In [1]:
#Imports

from marketsai.markets.diff_demand import DiffDemandDiscrete
from marketsai.economies.economies import Economy

#import ray

from ray import tune, shutdown, init
from ray.tune.registry import register_env
from ray.rllib.agents.a3c.a2c import A2CTrainer
from ray.rllib.agents.dqn.dqn import DQNTrainer
from ray.tune.integration.mlflow import MLflowLoggerCallback
from ray.rllib.utils.schedules.exponential_schedule import ExponentialSchedule

import random
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Instructions for updating:
non-resource variables are not supported in the long term
{'agent_0': [[7, 15], [7, 15]], 'agent_1': [[7, 15], [7, 15]]} {'agent_0': 0.7545845781661231, 'agent_1': 0.1915165233934397} {'agent_0': True, 'agent_1': False} {'agent_0': [1.4666666666666668, 1.4666666666666668], 'agent_1': [2.0, 2.0]}


In [2]:
# STEP 0: Inititialize ray

NUM_CPUS = 14
shutdown()
init(num_cpus=NUM_CPUS)

2021-04-04 19:52:43,584	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.202',
 'raylet_ip_address': '192.168.1.202',
 'redis_address': '192.168.1.202:44956',
 'object_store_address': '/tmp/ray/session_2021-04-04_19-52-42_713200_25867/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-04-04_19-52-42_713200_25867/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-04-04_19-52-42_713200_25867',
 'metrics_export_port': 60183,
 'node_id': '7dcc6a6c16a46f25f50d437f654e4e7cbe3ee0d623388f99a2fb72b8'}

In [3]:
# STEP 1: register environment

register_env("economy", Economy)
env = Economy()
policy_ids = ["policy_{}".format(i) for i in range(env.n_agents)]

In [4]:
# STEP 2: Experiment configuration

MAX_STEPS = 5 * 1000
PRICE_BAND_WIDE = 0.1
LOWER_PRICE = 1.47 - PRICE_BAND_WIDE
HIGHER_PRICE = 1.93 + PRICE_BAND_WIDE
DEC_RATE = math.e ** (-4 * 10 ** (-6))
DEC_RATE_HIGH = math.e ** (-4 * 10 ** (-6) * 4)
market_config = {
    "LOWER_PRICE": [LOWER_PRICE for i in range(env.n_agents)],
    "HIGHER_PRICE": [HIGHER_PRICE for i in range(env.n_agents)],
}
env_config = {"markets_dict": {"market_0": (DiffDemandDiscrete, market_config), "market_1": (DiffDemandDiscrete, market_config)}}

exploration_config = {
    "type": "EpsilonGreedy",
    "epsilon_schedule": ExponentialSchedule(
        schedule_timesteps=1,
        framework=None,
        initial_p=1,
        decay_rate=DEC_RATE,
    ),
}

config = {
    "gamma": 0.95,
    "lr": 0.15,
    "env": "economy",
    "exploration_config": exploration_config,
    "env_config": env_config,
    "horizon": 100,
    "soft_horizon": True,
    "no_done_at_end": True,
    "multiagent": {
        "policies": {
            policy_ids[i]: (
                None,
                env.observation_space["agent_{}".format(i)],
                env.action_space["agent_{}".format(i)],
                {},
            )
            for i in range(env.n_agents)
        },
        "policy_mapping_fn": (lambda agent_id: policy_ids[int(agent_id.split("_")[1])]),
    },
    "framework": "torch",
    "num_workers": NUM_CPUS - 1,
    "num_gpus": 0,
}

stop = {"info/num_steps_trained": MAX_STEPS}

In [5]:
#Step 3: Experiments

exp_name = "DQN_econ_April4"
results = tune.run(
    "DQN",
    name=exp_name,
    config=config,
    checkpoint_freq=250,
    checkpoint_at_end=True,
    stop=stop,
    metric="episode_reward_mean",
    mode="max",
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name, save_artifact=True)],
)

best_checkpoint = results.best_checkpoint
print("Best checkpont:", best_checkpoint)

Trial name,status,loc
DQN_economy_e31af_00000,RUNNING,


(pid=26092) /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
(pid=26092)   warnings.warn(msg)
(pid=26092) WARNING:tensorflow:From /Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=26092) Instructions for updating:
(pid=26092) non-resource variables are not supported in the long term
(pid=26092) 2021-04-04 19:53:02,824	INFO trainer.py:641 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=26092) {'agent_0': [[7, 15], [7, 15]], 'agent_1': [[7, 15]

Trial name,status,loc
DQN_economy_e31af_00000,ERROR,
Trial name,# failures,error file
DQN_economy_e31af_00000,1,/Users/matiascovarrubias/ray_results/DQN_econ_April4/DQN_economy_e31af_00000_0_2021-04-04_19-53-00/error.txt


Trial name,status,loc
DQN_economy_e31af_00000,ERROR,
Trial name,# failures,error file
DQN_economy_e31af_00000,1,/Users/matiascovarrubias/ray_results/DQN_econ_April4/DQN_economy_e31af_00000_0_2021-04-04_19-53-00/error.txt


as/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/tune/registry.py", line 122, in register
(pid=26080)     self._to_flush[(category, key)] = pickle.dumps_debug(value)
(pid=26080)   File "/Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/cloudpickle/__init__.py", line 39, in dumps_debug
(pid=26080)     return dumps(obj, *args, **kwargs)
(pid=26080)   File "/Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 73, in dumps
(pid=26080)     cp.dump(obj)
(pid=26080)   File "/Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-reVLCGV_-py3.8/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle_fast.py", line 574, in dump
(pid=26080)     return Pickler.dump(self, obj)
(pid=26080)   File "/Users/matiascovarrubias/Library/Caches/pypoetry/virtualenvs/marketsai-r

TuneError: ('Trials did not complete', [DQN_economy_e31af_00000])

In [ ]:
#Step 4: Evaluation

config["evaluation_config"] = {"explore": False}
trained_trainer = DQNTrainer(config=config)
trained_trainer.restore(best_checkpoint)
price_agent0_list = []
reward_agent0_list = []
price_agent1_list = []
reward_agent1_list = []
obs, reward, done, info = env.step({"agent_0": 1, "agent_1": 11})
for i in range(500):

    action_agent0 = trained_trainer.compute_action(obs["agent_0"], policy_id="policy_0")
    action_agent1 = trained_trainer.compute_action(obs["agent_1"], policy_id="policy_1")
    obs, reward, done, info = env.step(
        {"agent_0": action_agent0, "agent_1": action_agent1}
    )
    price_agent0_list.append(info["agent_0"])
    reward_agent0_list.append(reward["agent_0"])
    price_agent1_list.append(info["agent_1"])
    reward_agent1_list.append(reward["agent_1"])

plt.ion()

plt.plot(price_agent0_list)
plt.show()
plt.plot(price_agent1_list)
plt.show()

IRresults = {
    "Profits Agent 0": reward_agent0_list,
    "Profits Agent 1": reward_agent1_list,
    "Price Agent 0": price_agent0_list,
    "Price Agent 1": price_agent1_list,
}
df_IR = pd.DataFrame(IRresults)
df_IR.to_csv("collusion_IR_DQN.csv")